In [1]:
!pip -q install pandas openpyxl requests


In [2]:
import io, requests, pandas as pd

RAW_URL = "https://raw.githubusercontent.com/Krzysztof-Broniszewski/codebrainers-plotly-dash/main/Wypadki_2004_2023_KGP.xlsx"

content = requests.get(RAW_URL, timeout=20).content
raw = pd.read_excel(io.BytesIO(content), header=None, engine="openpyxl")
raw.head(10)


,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Wypadki drogowe w Polsce,NaN,NaN,NaN,NaN,Wypadki z winy kierującego pojazdem,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Rok,Wypadki drogowe,Ofiary śmiertelne,Ranni,NaN,Rok,Wypadki drogowe,Ofiary śmiertelne,Ranni
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,2004,51069,5712,64661,NaN,2005,39730,4239,53429
8,NaN,2005,48100,5444,61191,NaN,2006,37129,3729,49784
9,NaN,2006,46876,5243,59123,NaN,2007,38434,3753,52240


In [3]:
raw.shape

(30, 10)

In [4]:
!pip -q install unidecode

In [5]:
import re
from unidecode import unidecode

# 1) znajdź wiersz z nagłówkami "Rok | Wypadki | Ofiary | Ranni"
header_row = None
for i in range(min(30, len(raw))):
    row = raw.iloc[i].astype(str).str.strip()
    if row.str.fullmatch(r"[Rr]ok").any() and row.str.contains(r"wypad|ofiary|rann", case=False, regex=True).any():
        header_row = i
        break
if header_row is None:
    header_row = 0

hdr = raw.iloc[header_row].fillna("").astype(str).str.strip()

# 2) pozycje kolumn "Rok" (lewy i prawy blok)
rok_pos = sorted([j for j, v in hdr.items() if v.lower() == "rok"])

blocks = []
for idx, start in enumerate(rok_pos):
    # mapowanie wg pozycji: 0 = lewy (W Polsce), 1 = prawy (Z winy kierującego)
    label = "W Polsce" if idx == 0 else "Z winy kierującego"

    cols = [start, start+1, start+2, start+3]           # Rok + 3 metryki
    cols = [c for c in cols if c < raw.shape[1]]        # bez wyjścia poza zakres
    part = raw.iloc[header_row+1:, cols].copy()         # dane pod nagłówkiem
    # ustaw prawdziwe nazwy kolumn z wiersza nagłówka
    part.columns = raw.iloc[header_row, cols].astype(str).str.strip().tolist()
    part["Kategoria"] = label
    blocks.append(part)

# 3) sklej i ujednolić nazwy kolumn
df = pd.concat(blocks, axis=0, ignore_index=True)

# dopasuj nazwy (niektóre XLSX mogą mieć różne warianty)
def by_contains(name_substr):
    return next(c for c in df.columns if name_substr in c.lower())

df = df.rename(columns={
    next(c for c in df.columns if c.lower() == "rok"): "Rok",
    by_contains("wypad"): "Wypadki drogowe",
    by_contains("ofiary"): "Ofiary smiertelne",
    by_contains("rann"): "Ranni",
})

# 4) konwersje do liczb
def to_num(series):
    s = (series.astype(str)
                 .str.replace(r"\s", "", regex=True)
                 .str.replace(",", ".", regex=False))
    return pd.to_numeric(s, errors="coerce")

df["Rok"] = pd.to_numeric(df["Rok"], errors="coerce")
for c in ["Wypadki drogowe", "Ofiary smiertelne", "Ranni"]:
    df[c] = to_num(df[c])

# 5) porządki: zachowujemy OBA bloki (bez drop_duplicates po "Rok")
df = (df.dropna(subset=["Rok"])
        .astype({"Rok": int})
        .sort_values(["Rok", "Kategoria"])
        .reset_index(drop=True))

df.head(), df["Kategoria"].value_counts(), df.dtypes


(    Rok  Wypadki drogowe  Ofiary smiertelne    Ranni           Kategoria
 0  2004          51069.0             5712.0  64661.0            W Polsce
 1  2005          48100.0             5444.0  61191.0            W Polsce
 2  2005          39730.0             4239.0  53429.0  Z winy kierującego
 3  2006          46876.0             5243.0  59123.0            W Polsce
 4  2006          37129.0             3729.0  49784.0  Z winy kierującego,
 Kategoria
 W Polsce              20
 Z winy kierującego    19
 Name: count, dtype: int64,
 Rok                    int64
 Wypadki drogowe      float64
 Ofiary smiertelne    float64
 Ranni                float64
 Kategoria             object
 dtype: object)

In [6]:
!pip -q install jupyter-dash dash plotly


In [7]:
from jupyter_dash import JupyterDash
from dash import html, dcc, Dash, Input, Output
import plotly.express as px


In [8]:
px.defaults.template = "plotly_dark"  # (ciemny, jak lubisz)
fig = px.line(
    df,
    x="Rok",
    y="Wypadki drogowe",
    color="Kategoria",  # dzięki temu zobaczysz dwie serie
    markers=True,
    title="Wypadki drogowe – porównanie kategorii"
)
fig.show()


In [9]:
px.defaults.template = "plotly_dark"
fig = px.line(df, x="Rok", y="Wypadki drogowe", color="Kategoria", markers=True)

metrics = ["Wypadki drogowe", "Ofiary smiertelne", "Ranni"]

app = Dash(__name__)
app.layout = html.Div([
    html.H3("Demo: wypadki drogowe"),
    dcc.RadioItems(
        id="metric",
        options=[{"label": m, "value": m} for m in metrics if m in df.columns],
        value=[m for m in metrics if m in df.columns][0],
        inline=True
    ),
    dcc.Graph(id="g1")
])


In [10]:
@app.callback(
    Output("g1", "figure"),          # WYJŚCIE: co aktualizujemy
    Input("metric", "value")         # WEJŚCIE: co śledzimy
)
def update_chart(selected_metric):
    # obrona na wypadek None
    if not selected_metric or selected_metric not in df.columns:
        return px.line()  # pusty wykres

    fig = px.line(
        df,
        x="Rok",
        y=selected_metric,
        color="Kategoria",     # dwie serie: W Polsce vs Z winy kierującego
        markers=True,
        title=f"{selected_metric} — porównanie kategorii"
    )
    fig.update_layout(transition_duration=300)
    return fig


In [11]:
import threading, time
from google.colab.output import eval_js

PORT = 8050  # zmień np. na 8060 jeśli zajęty

def _run():
    # w nowszym Dash używaj app.run(), bez reloadera
    app.run(host="0.0.0.0", port=PORT, debug=False, use_reloader=False)

t = threading.Thread(target=_run, daemon=True)
t.start()

# poczekaj aż wstanie i pobierz publiczny URL od Colaba
url = None
for _ in range(40):  # ~20s na start
    try:
        url = eval_js(f"google.colab.kernel.proxyPort({PORT})")
        if url: break
    except Exception:
        pass
    time.sleep(0.5)

print("URL:", url or "nie uzyskano (zmień PORT i spróbuj ponownie)")

# otwórz w nowej karcie
if url:
    eval_js(f"window.open('{url}', '_blank')")


<IPython.core.display.Javascript object>

URL: https://8050-m-s-3fi97bzrw1prd-b.us-west1-1.prod.colab.dev
